## 📚 Assignment: Transfer Learning & The Power of Initialization
## Building Intuition for MAML

**Learning Objectives:**
- Understand why initialization matters for few-shot learning
- Experience the difference between various pre-training strategies
- Develop intuition for what MAML tries to optimize

**Advice on using LLM's**

---


Avoid it , but unfortunately we cannot stop you from using it , dont ask it everything more you think on your own the better , but whenever you take in a code from it , understand how that part fits in the current code , is there some optimization it did on its own, node it down or comment it in the code.

In [ ]:
!pip install -q torch torchvision matplotlib numpy

#Understand what does each of this import do , see what all functions this hold
#whenever you want to implement something think which of this would you use and refer to its doc for the syntax

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import random

print("✅ Setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

✅ Setup complete!
PyTorch version: 2.9.0+cpu
CUDA available: False
Using device: cpu


## 📊 Part A: Dataset Preparation

We'll use **MNIST** for simplicity (or you can use Omniglot if you prefer).

**Your Task:**
- Split MNIST into 5 tasks (Tasks A-E), each with 2 digit classes
- For example: Task A = {0, 1}, Task B = {2, 3}, etc.

In [ ]:
# think on the architecture of the model as discussed in class
#general flow -> convolution->relu->maxpooling and so on , in the end some fully connected layers then final classification
# Refer to the 60 minute pytorch implementation section of 'neural networks'


#Implement the class or the model here
#fill in the objects(layers) and methods(forward pass)

Now since the model is ready we decide how do we want to train it :

First Do normal classification on large dataset of Task A - 0 & 1.

The we will do fine tuning

1.   Random Initialisation and then fine tune using support dataset, say we do this for task A which were 0 & 1 digits (save this)
2.   Take the above model weights and fine tune it on the support dataset for some other task , say B(2's & 3's)
3.   First train the model on all combined train dataset for all 10 digits(from all tasks A,B,C,D,E), then save it and then fine tune it on support dataset on to make a binary classifier , any 1 task say A here now digits will be classified. 0 class->0 digit , 1->1.

While moving from one model to other , think what layers do i need to keep and what do i need to remove.



In [ ]:
#Method 1 - Fully Trained Binary Classifier on Task A

In [ ]:
#Method 2 - Fine Tuning all 3 methods

At the end compare performance of all this models and methods using the Query Set.

Also plot the learning curve vs epoch for all the methods

Make a table and fill in the values of different evaluation metrics you learned in previous lectures.

In [ ]:
#Analysis and Plots

Some Theoritical Questions :

1.   Which strategy in Method 2 works best and why do you feel so ?
2.   In Part 3 of Method 2 we have trained the model already on Task B as well when we made a 10 class classifier, then when we are fine tuning it again using support set what exactly is happening ?
3.   What if we used the 10 digit classifier to make a binary classifier for a binary letter classification , will it work or rather how will you make it work ?
4.   Where exactly have we used Meta Learning, in which approach? Have we even used it ?

---


Digit classifier and letter classifier are two dissimilar tasks can we have starting point or a initialisation such that when we fine tuning using a few datapoints for both tasks we get optmimal result ? This is what we will try to do in MAML ?


---


Think on them sincerely , would love to read your answers!



In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(), # Converts PIL Image or numpy.ndarray to tensor
    transforms.Normalize((0.1307,), (0.3081,)) # Normalize with MNIST mean and std
])

train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)
# we get a special parameter while loading which is 'background'
#refer to document for what it means and how to use it
# NOTE: The 'background' parameter is not a standard parameter for torchvision.datasets.MNIST.
# It has been omitted from the dataset loading calls.

print(f"✅ MNIST loaded: {len(train_dataset)} train, {len(test_dataset)} test images")

# TODO: Define your task structure
# We'll split 10 digits into 5 tasks, each with 2 classes

100%|██████████| 9.91M/9.91M [00:00<00:00, 147MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 37.3MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 69.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.26MB/s]


✅ MNIST loaded: 60000 train, 10000 test images


In [ ]:
# TODO: Define your task structure
# We'll split 10 digits into 5 tasks, each with 2 classes

task_definitions = {
    'A': [0, 1],
    'B': [2, 3],
    'C': [4, 5],
    'D': [6, 7],
    'E': [8, 9]
}

#Below function should take the given inputs and split the main dataset with the given input classes into train,support and query.
def create_task_datasets(dataset, task_classes, n_train=15, n_support=5, n_query=10):
    """
    Create train, support, and query sets for a specific task.

    Args:
        dataset: Full MNIST dataset
        task_classes: List of class labels for this task [e.g., [0, 1]]
        n_train: Number of training examples per class
        n_support: Number of support examples per class (for fine-tuning)
        n_query: Number of query examples per class (for testing)

    Returns:
        train_data, support_data, query_data (each is list of (image, label) tuples
    """
# TODO: Implement this function
# HINT: Filter dataset to only include examples from task_classes
# HINT: Split into train/support/query sets

In [ ]:
task_definitions = {
    'A': [0, 1],
    'B': [2, 3],
    'C': [4, 5],
    'D': [6, 7],
    'E': [8, 9]
}

def create_task_datasets(dataset, task_classes, n_train=15, n_support=5, n_query=10):
    """
    Create train, support, and query sets for a specific task.

    Args:
        dataset: Full MNIST dataset
        task_classes: List of class labels for this task [e.g., [0, 1]]
        n_train: Number of training examples per class
        n_support: Number of support examples per class (for fine-tuning)
        n_query: Number of query examples per class (for testing)

    Returns:
        train_data, support_data, query_data (each is list of (image, label) tuples
    """
    task_data_by_class = defaultdict(list)
    for img, label in dataset:
        if label in task_classes:
            task_data_by_class[label].append((img, label))

    train_data = []
    support_data = []
    query_data = []

    for class_label in task_classes:
        class_samples = task_data_by_class[class_label]
        random.shuffle(class_samples)

        # Ensure we have enough samples for each set
        if len(class_samples) < (n_train + n_support + n_query):
            # Handle cases where there aren't enough samples for specified counts
            # For now, we'll just take what's available up to the limit
            print(f"Warning: Not enough samples for class {class_label}. Required: {n_train + n_support + n_query}, Available: {len(class_samples)}")

        train_data.extend(class_samples[:n_train])
        support_data.extend(class_samples[n_train : n_train + n_support])
        query_data.extend(class_samples[n_train + n_support : n_train + n_support + n_query])

    return train_data, support_data, query_data

In [ ]:
train_A, support_A, query_A = create_task_datasets(train_dataset, task_definitions['A'])
print(f"Task A - Train: {len(train_A)}, Support: {len(support_A)}, Query: {len(query_A)}")

Task A - Train: 30, Support: 10, Query: 20


In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        # First convolutional block
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # Input channels: 1 (MNIST is grayscale)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second convolutional block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        # Calculate the size of the flattened features
        # Input image size: 28x28
        # After pool1: (28/2) x (28/2) = 14x14
        # After pool2: (14/2) x (14/2) = 7x7
        # Number of channels after conv2: 64
        self.fc1 = nn.Linear(64 * 7 * 7, 128) # 64 channels * 7 * 7 spatial dimensions
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7) # Flatten the output for the fully connected layers
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

print("✅ CNN model class `Net` defined successfully!")

✅ CNN model class `Net` defined successfully!


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        # Assuming images are already tensors and labels are integers
        # If labels need to be adjusted (e.g., 0 and 1 instead of 0 and 1 for specific tasks),
        # this is where you'd do it. For Task A (0,1), it's straightforward.
        return image, label

# 1. Instantiate the Net model with num_classes=2
model_method1 = Net(num_classes=2).to(device)

# 2. Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_method1.parameters(), lr=0.001)

# 3. Create a DataLoader for the train_A dataset
train_dataset_method1 = CustomDataset(train_A)
train_loader_method1 = DataLoader(train_dataset_method1, batch_size=16, shuffle=True)

# Parameters for training
epochs = 10

# Lists to store training metrics
train_losses_method1 = []
train_accuracies_method1 = []

print(f"\n🚀 Starting training for Method 1 on Task A for {epochs} epochs...")

# 4. Train the model
for epoch in range(epochs):
    model_method1.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for i, (inputs, labels) in enumerate(train_loader_method1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_method1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader_method1)
    epoch_accuracy = 100 * correct_predictions / total_predictions
    train_losses_method1.append(epoch_loss)
    train_accuracies_method1.append(epoch_accuracy)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

print("✅ Training for Method 1 complete!")

# 6. Save the state dictionary of the trained model
torch.save(model_method1.state_dict(), 'model_method1.pth')
print("Model weights for Method 1 saved to 'model_method1.pth'")



🚀 Starting training for Method 1 on Task A for 10 epochs...
Epoch 1/10, Loss: 0.5870, Accuracy: 70.00%
Epoch 2/10, Loss: 0.2199, Accuracy: 100.00%
Epoch 3/10, Loss: 0.0508, Accuracy: 100.00%
Epoch 4/10, Loss: 0.0308, Accuracy: 100.00%
Epoch 5/10, Loss: 0.0034, Accuracy: 100.00%
Epoch 6/10, Loss: 0.0014, Accuracy: 100.00%
Epoch 7/10, Loss: 0.0041, Accuracy: 100.00%
Epoch 8/10, Loss: 0.0031, Accuracy: 100.00%
Epoch 9/10, Loss: 0.0014, Accuracy: 100.00%
Epoch 10/10, Loss: 0.0002, Accuracy: 100.00%
✅ Training for Method 1 complete!
Model weights for Method 1 saved to 'model_method1.pth'


In [ ]:
model_method2_1 = Net(num_classes=2).to(device)

criterion_2_1 = nn.CrossEntropyLoss()
optimizer_2_1 = optim.Adam(model_method2_1.parameters(), lr=0.001)

support_dataset_2_1 = CustomDataset(support_A)
support_loader_2_1 = DataLoader(support_dataset_2_1, batch_size=len(support_A), shuffle=True)

epochs_2_1 = 50 # Fine-tuning usually requires fewer epochs or more targeted training

train_losses_method2_1 = []
train_accuracies_method2_1 = []

print(f"\n🚀 Starting fine-tuning for Method 2.1 (Random Init + Task A Support) for {epochs_2_1} epochs...")

for epoch in range(epochs_2_1):
    model_method2_1.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in support_loader_2_1:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_2_1.zero_grad()

        outputs = model_method2_1(inputs)
        loss = criterion_2_1(outputs, labels)
        loss.backward()
        optimizer_2_1.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(support_loader_2_1)
    epoch_accuracy = 100 * correct_predictions / total_predictions
    train_losses_method2_1.append(epoch_loss)
    train_accuracies_method2_1.append(epoch_accuracy)

    if (epoch + 1) % 10 == 0 or epoch == epochs_2_1 - 1:
        print(f"Epoch {epoch+1}/{epochs_2_1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

print("✅ Fine-tuning for Method 2.1 complete!")

torch.save(model_method2_1.state_dict(), 'model_method2_1.pth')
print("Model weights for Method 2.1 saved to 'model_method2_1.pth'")


🚀 Starting fine-tuning for Method 2.1 (Random Init + Task A Support) for 50 epochs...
Epoch 10/50, Loss: 0.0036, Accuracy: 100.00%
Epoch 20/50, Loss: 0.0000, Accuracy: 100.00%
Epoch 30/50, Loss: 0.0000, Accuracy: 100.00%
Epoch 40/50, Loss: 0.0000, Accuracy: 100.00%
Epoch 50/50, Loss: 0.0000, Accuracy: 100.00%
✅ Fine-tuning for Method 2.1 complete!
Model weights for Method 2.1 saved to 'model_method2_1.pth'


In [ ]:
print("\n--- Method 2.2: Transfer from Task A to Task B ---")

# 1. Generate support_B and query_B datasets
train_B, support_B, query_B = create_task_datasets(train_dataset, task_definitions['B'])
print(f"Task B - Train: {len(train_B)}, Support: {len(support_B)}, Query: {len(query_B)}")

# 2. Instantiate a *new* Net model for 2 classes
model_method2_2 = Net(num_classes=2).to(device)

# 3. Load the state dictionary from model_method1.pth (pre-trained on Task A)
try:
    model_method2_2.load_state_dict(torch.load('model_method1.pth'))
    print("✅ Loaded weights from 'model_method1.pth' successfully.")
except Exception as e:
    print(f"❌ Error loading model_method1.pth: {e}")
    print("Ensure Method 1 was executed and the model was saved.")

# 2.1. Remap labels in support_B (2 and 3) to 0 and 1
# This creates a new list of (image, remapped_label) tuples
remapped_support_B = []
for img, label in support_B:
    if label == 2:
        remapped_support_B.append((img, 0))
    elif label == 3:
        remapped_support_B.append((img, 1))
# Also remap query_B for evaluation later
remapped_query_B = []
for img, label in query_B:
    if label == 2:
        remapped_query_B.append((img, 0))
    elif label == 3:
        remapped_query_B.append((img, 1))

# 4. Define a loss function and an optimizer (smaller learning rate for fine-tuning)
criterion_2_2 = nn.CrossEntropyLoss()
optimizer_2_2 = optim.Adam(model_method2_2.parameters(), lr=0.0001) # Smaller LR

# 5. Create a DataLoader for the remapped support_B dataset
support_dataset_2_2 = CustomDataset(remapped_support_B)
support_loader_2_2 = DataLoader(support_dataset_2_2, batch_size=len(remapped_support_B), shuffle=True)

epochs_2_2 = 50 # Fine-tuning usually requires fewer epochs or more targeted training

train_losses_method2_2 = []
train_accuracies_method2_2 = []

print(f"🚀 Starting fine-tuning for Method 2.2 (Transfer from A to B) for {epochs_2_2} epochs...")

# 6. Train the model for a few epochs on the support_B dataset
for epoch in range(epochs_2_2):
    model_method2_2.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in support_loader_2_2:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_2_2.zero_grad()

        outputs = model_method2_2(inputs)
        loss = criterion_2_2(outputs, labels)
        loss.backward()
        optimizer_2_2.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(support_loader_2_2)
    epoch_accuracy = 100 * correct_predictions / total_predictions
    train_losses_method2_2.append(epoch_loss)
    train_accuracies_method2_2.append(epoch_accuracy)

    if (epoch + 1) % 10 == 0 or epoch == epochs_2_2 - 1:
        print(f"Epoch {epoch+1}/{epochs_2_2}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

print("✅ Fine-tuning for Method 2.2 complete!")

# 8. Save the model's state dictionary
torch.save(model_method2_2.state_dict(), 'model_method2_2.pth')
print("Model weights for Method 2.2 saved to 'model_method2_2.pth'")


--- Method 2.2: Transfer from Task A to Task B ---
Task B - Train: 30, Support: 10, Query: 20
✅ Loaded weights from 'model_method1.pth' successfully.
🚀 Starting fine-tuning for Method 2.2 (Transfer from A to B) for 50 epochs...
Epoch 10/50, Loss: 0.8752, Accuracy: 70.00%
Epoch 20/50, Loss: 0.5991, Accuracy: 80.00%
Epoch 30/50, Loss: 0.3114, Accuracy: 80.00%
Epoch 40/50, Loss: 0.1274, Accuracy: 100.00%
Epoch 50/50, Loss: 0.0616, Accuracy: 100.00%
✅ Fine-tuning for Method 2.2 complete!
Model weights for Method 2.2 saved to 'model_method2_2.pth'


In [ ]:
print("\n--- Method 2.3: Pre-training on All Digits + Fine-tuning on Task A Support Set ---")

# 1. Create a combined dataset of all 10 digits from the train_dataset
# The original 'train_dataset' already contains all 10 digits, so we can use it directly.
full_train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 2. Instantiate a new Net model with num_classes=10 (for the pre-training phase)
model_pretrained_all_digits = Net(num_classes=10).to(device)

# Define loss and optimizer for pre-training
criterion_pretrain = nn.CrossEntropyLoss()
optimizer_pretrain = optim.Adam(model_pretrained_all_digits.parameters(), lr=0.001)

epochs_pretrain = 5 # Pre-train for a few epochs

print(f"\n🚀 Starting pre-training for Method 2.3 (All Digits) for {epochs_pretrain} epochs...")

# Lists to store training metrics for pre-training
pretrain_losses = []
pretrain_accuracies = []

# 3. Train this 10-class classifier on the combined dataset
for epoch in range(epochs_pretrain):
    model_pretrained_all_digits.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in full_train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_pretrain.zero_grad()

        outputs = model_pretrained_all_digits(inputs)
        loss = criterion_pretrain(outputs, labels)
        loss.backward()
        optimizer_pretrain.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(full_train_loader)
    epoch_accuracy = 100 * correct_predictions / total_predictions
    pretrain_losses.append(epoch_loss)
    pretrain_accuracies.append(epoch_accuracy)

    print(f"Pre-train Epoch {epoch+1}/{epochs_pretrain}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

print("✅ Pre-training for Method 2.3 complete!")

# Save pre-trained model weights
torch.save(model_pretrained_all_digits.state_dict(), 'model_pretrained_all_digits.pth')
print("Pre-trained model weights saved to 'model_pretrained_all_digits.pth'")

# --- Fine-tuning Phase ---
print("\n🚀 Starting fine-tuning for Method 2.3 on Task A Support Set...")

# 4. Instantiate another Net model for num_classes=2 (for fine-tuning on Task A)
model_method2_3 = Net(num_classes=2).to(device)

# 5. Load the weights from model_pretrained_all_digits.pth into the feature extraction layers
#    We load all layers except the final classification layer (fc2) as its output size differs.
pretrained_state_dict = torch.load('model_pretrained_all_digits.pth')
filtered_state_dict = {k: v for k, v in pretrained_state_dict.items() if 'fc2' not in k}
model_method2_3.load_state_dict(filtered_state_dict, strict=False)
print("✅ Loaded pre-trained weights (excluding final classifier) for fine-tuning.")

# Optional: Freeze feature extraction layers, only train the new head
# for param in model_method2_3.parameters():
#     param.requires_grad = False
# for param in model_method2_3.fc2.parameters():
#     param.requires_grad = True # Ensure the new head is trainable

# 6. Define a new loss function and optimizer for the fine-tuning phase
criterion_2_3 = nn.CrossEntropyLoss()
optimizer_2_3 = optim.Adam(model_method2_3.parameters(), lr=0.00005) # Very small LR for fine-tuning

# 7. Create a DataLoader for the support_A dataset (labels 0 and 1 are already correct)
support_dataset_2_3 = CustomDataset(support_A)
support_loader_2_3 = DataLoader(support_dataset_2_3, batch_size=len(support_A), shuffle=True)

epochs_2_3 = 50 # Fine-tune for a few epochs

# Lists to store training metrics for fine-tuning
train_losses_method2_3 = []
train_accuracies_method2_3 = []

# 8. Train the 2-class model on the support_A dataset
for epoch in range(epochs_2_3):
    model_method2_3.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in support_loader_2_3:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_2_3.zero_grad()

        outputs = model_method2_3(inputs)
        loss = criterion_2_3(outputs, labels)
        loss.backward()
        optimizer_2_3.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(support_loader_2_3)
    epoch_accuracy = 100 * correct_predictions / total_predictions
    train_losses_method2_3.append(epoch_loss)
    train_accuracies_method2_3.append(epoch_accuracy)

    if (epoch + 1) % 10 == 0 or epoch == epochs_2_3 - 1:
        print(f"Fine-tune Epoch {epoch+1}/{epochs_2_3}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

print("✅ Fine-tuning for Method 2.3 complete!")

# 10. Save the final fine-tuned model's state dictionary
torch.save(model_method2_3.state_dict(), 'model_method2_3.pth')
print("Model weights for Method 2.3 saved to 'model_method2_3.pth'")



--- Method 2.3: Pre-training on All Digits + Fine-tuning on Task A Support Set ---

🚀 Starting pre-training for Method 2.3 (All Digits) for 5 epochs...
Pre-train Epoch 1/5, Loss: 0.1316, Accuracy: 95.97%
Pre-train Epoch 2/5, Loss: 0.0432, Accuracy: 98.69%
Pre-train Epoch 3/5, Loss: 0.0292, Accuracy: 99.07%
Pre-train Epoch 4/5, Loss: 0.0203, Accuracy: 99.34%




1. Pretraining on all 10 digits and then fine-tuning works best because the model learns general digit features first and only needs small adjustments for a specific task, giving better accuracy with less data.


2. Fine-tuning sharpens the model’s decision boundary for only the required classes, adapting general knowledge to a specific binary task.


3. Directly no, but it can work by keeping early convolution layers, replacing the final classifier, and fine-tuning on letter data, since low-level visual features are transferable.


4. We have not truly used meta-learning. We only used transfer learning and fine-tuning, not a meta-learning algorithm like MAML.




